In [21]:
import sys
import os

# Add the src directory to sys.path
src_path = os.path.abspath(os.path.join('..', 'src'))
if src_path not in sys.path:
    sys.path.append(src_path)


In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ruptures as rpt
import pymc as pm
import arviz as az 

ModuleNotFoundError: No module named 'pymc'

In [ ]:
print(df.columns)

In [ ]:
import pymc as pm
import aesara.tensor as at
import numpy as np

data = np.random.randn(100)
n = len(data)

with pm.Model() as model:
    tau = pm.DiscreteUniform("tau", lower=0, upper=n)
    mu1 = pm.Normal("mu1", mu=0, sigma=1)
    mu2 = pm.Normal("mu2", mu=0, sigma=1)
    
    mu = pm.math.switch(tau >= at.arange(n), mu1, mu2)
    obs = pm.Normal("obs", mu=mu, sigma=1, observed=data)

    trace = pm.sample(
    draws=2000,
    tune=2000,
    target_accept=0.98,  # very cautious step size
    return_inferencedata=True
             )

    pm.summary(trace)


In [ ]:
import arviz as az
az.summary(trace, hdi_prob=0.95)

In [ ]:
az.plot_trace(trace)
plt.show()

az.plot_posterior(trace)
plt.show()


In [ ]:
tau_samples = trace.posterior['tau'].values.flatten()
estimated_tau = int(np.mean(tau_samples))
print("Estimated change point (index):", estimated_tau)


In [ ]:
path = "../data/processed/brent_clean.csv"

try:
    data = pd.read_csv(path)

    # Use format='mixed' to infer the format for each date string
    data['Date'] = pd.to_datetime(data['Date'], format='mixed', dayfirst=False)
    data = data.sort_values(by='Date')
    data = data.set_index('Date')

    # Handle missing values by interpolation, as suggested for robust analysis
    # This is crucial for time series data to maintain continuity
    data['Price'] = data['Price'].interpolate(method='time')

    # Drop any rows where 'Price' might still be NaN after interpolation (e.g., at the very beginning/end if no surrounding data)
    data = data.dropna(subset=['Price'])

    # For Bayesian analysis of volatility, it's often better to work with log returns
    # Log returns help in achieving stationarity and are directly related to volatility.
    # We use.diff() and.dropna() to handle the first NaN value from the difference calculation.
    data['Log_Return'] = np.log(data['Price']).diff().dropna()

    # The time series for analysis will be the 'Price' for ruptures and 'Log_Return' for PyMC
    time_series_price = data['Price']
    time_series_log_return = data['Log_Return'].dropna() # Ensure log returns are also dropped if NaN

    print("Brent Oil Prices Time Series Head:")
    display(time_series_price.head())
    print("\nBrent Oil Prices Time Series Tail:")
    display(time_series_price.tail())
    print("\nBrent Oil Log Returns Time Series Head:")
    display(time_series_log_return.head())
    print("\nBrent Oil Log Returns Time Series Tail:")
    display(time_series_log_return.tail())

except FileNotFoundError:
    print(f"Error: The file '{path}' was not found. Please update the 'path' variable with the correct file location.")

except Exception as e:
    print(f"An error occurred during data loading or preprocessing: {e}")


In [ ]:
print(type(data))


In [ ]:
data = df[['Date', 'daily_return']].values

In [ ]:
data = df['daily_return'].values

In [ ]:
print(data.shape)

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(df['Date'], df['daily_return'], label='Daily Return')
plt.axvline(x=df['Date'].iloc[estimated_tau], color='red', linestyle='--', label='Estimated Change Point')
plt.xlabel('Date')
plt.ylabel('Daily Return')
plt.title('Change Point Detection')
plt.legend()
plt.tight_layout()
import os

# Create the directory if it doesn't exist
os.makedirs('reports/figures', exist_ok=True)

# Then save the figure
plt.savefig('reports/figures/change_point_plot.png')

plt.savefig('reports/figures/change_point_plot.png')
plt.show()



In [ ]:
# Convert time series to a numpy array for the ruptures library
# We use the raw price data here to detect shifts in the mean price level.
if 'time_series_price' in locals() and not time_series_price.empty: # Check if time_series_price was successfully created and is not empty
    signal_array_ruptures = time_series_price.values.reshape(-1, 1) # Reshape for ruptures

    # Define the model and algorithm
    # 'l2' (L2 norm) cost function is good for detecting changes in mean.
    model_ruptures = "l2"
    algo_ruptures = rpt.Pelt(model=model_ruptures).fit(signal_array_ruptures)

    # Predict change points
    # The 'pen' (penalty) parameter controls the number of detected change points.
    # A higher penalty results in fewer change points.
    # You might need to tune this penalty for your specific dataset.
    # A common heuristic is 3 * log(n_samples) or 4 * log(n_samples).

    penalty_value_ruptures = 3 * np.log(len(time_series_price))
    change_points_ruptures = algo_ruptures.predict(pen=penalty_value_ruptures)

    # The 'ruptures' library returns the index of the last point of each segment.
    # We usually exclude the last point if it's just the end of the series.
    if change_points_ruptures and change_points_ruptures[-1] == len(time_series_price):
        detected_change_point_indices_ruptures = change_points_ruptures[:-1]
    else:
        detected_change_point_indices_ruptures = change_points_ruptures

    print(f"\nDetected Change Point Indices (ruptures on Price): {detected_change_point_indices_ruptures}")

    # Convert indices to dates for plotting
    detected_change_point_dates_ruptures = time_series_price.index[detected_change_point_indices_ruptures]
    print(f"Detected Change Point Dates (ruptures on Price): {detected_change_point_dates_ruptures.tolist()}")

    # Visualization for ruptures
    plt.figure(figsize=(18, 8))
    plt.plot(time_series_price.index, time_series_price.values, label='Brent Oil Price', color='blue', alpha=0.7)
    # Plot change points with labels, ensuring only one label is created for the legend
    for i, cp_date in enumerate(detected_change_point_dates_ruptures):
        plt.axvline(x=cp_date, color='red', linestyle='--', linewidth=2, label='Detected Change Point (ruptures)' if i == 0 else "")
    plt.title('Frequentist Change Point Detection (ruptures - PELT) on Brent Oil Price')
    plt.xlabel('Date')
    plt.ylabel('Price (USD)')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()
else:
    print("Skipping Frequentist Change Point Detection as price time series data is not available or empty.")

In [ ]:
# For Bayesian methods, we often define a probabilistic model.
# Let's model a single change point where the *volatility* (standard deviation) changes.
# This is applied to the log returns, as volatility is more directly observed there.

if 'time_series_log_return' in locals() and not time_series_log_return.empty: # Check if log returns were successfully created and are not empty
    # Convert log returns to numpy array for PyMC
    data_pymc_log_return = time_series_log_return.values

    # Define the Bayesian model
    with pm.Model() as bayesian_volatility_change_point_model:
        # Prior for the change point location (uniform over the time series)
        # We choose a discrete uniform distribution for the index of the change point.
        tau = pm.DiscreteUniform("tau", lower=0, upper=len(data_pymc_log_return) - 1)

        # Priors for the standard deviations (volatility) before and after the change point
        # We use HalfNormal distributions for standard deviations, as they must be positive.
        # Adjust sigma values based on expected scale of log returns.
        sigma_1 = pm.HalfNormal("sigma_1", sigma=0.1) # Volatility before change
        sigma_2 = pm.HalfNormal("sigma_2", sigma=0.1) # Volatility after change

        # Assume the mean of log returns is constant (e.g., around zero for daily returns)
        # or you could model it as well if you expect mean shifts in returns.
        # For simplicity, let's assume a constant mean for log returns.
        # A small normal prior around 0 is often reasonable for log returns.
        mu_log_return = pm.Normal("mu_log_return", mu=0, sigma=0.01)

        # Define the standard deviation for each data point based on the change point location (tau)
        idx = np.arange(len(data_pymc_log_return))
        sigma = pm.math.switch(idx < tau, sigma_1, sigma_2)

        # Likelihood of the observed data (log returns)
        # The observed log returns are assumed to be normally distributed around `mu_log_return` with standard deviation `sigma`.
        observation = pm.Normal("observation", mu=mu_log_return, sigma=sigma, observed=data_pymc_log_return)

        # Sample from the posterior distribution using NUTS (No-U-Turn Sampler)
        # This is where the MCMC magic happens.
        # `draws`: number of samples from the posterior.
        # `tune`: number of tuning (warmup) steps.
        # `chains`: number of independent sampling chains.
        # `cores`: number of CPU cores to use for sampling (set to 1 for simpler debugging, or higher for speed).
        # Sampling can take a while for large datasets.
        print("\nStarting PyMC sampling for Bayesian Volatility Change Point Detection...")
        trace_volatility = pm.sample(draws=4000, tune=3000, chains=4, random_seed=42, return_inferencedata=True, cores=2)
        print("PyMC Sampling Complete.")

    print("\nBayesian Volatility Model Sampling Complete. Summary of parameters:")
    display(az.summary(trace_volatility, var_names=["tau", "mu_log_return", "sigma_1", "sigma_2"]))

    # Analyze the posterior distribution of the change point (tau)
    tau_samples_volatility = trace_volatility.posterior["tau"].values.flatten()

    # Calculate the most probable change point index (mode of the posterior)
    # Note: PyMC's tau is an index relative to the `data_pymc_log_return` array, which starts after the first log return.
    # So, we need to adjust the index for the original `time_series_log_return` index.
    most_probable_tau_index_volatility = int(pd.Series(tau_samples_volatility).mode().iloc[0])
    # The log_return series starts one day after the price series, so adjust index for original date mapping
    most_probable_tau_date_volatility = time_series_log_return.index[most_probable_tau_index_volatility]

    print(f"\nMost Probable Change Point Index (PyMC on Log Returns): {most_probable_tau_index_volatility}")
    print(f"Most Probable Change Point Date (PyMC on Log Returns): {most_probable_tau_date_volatility}")

    # Visualization for PyMC results
    plt.figure(figsize=(18, 8))
    plt.plot(time_series_log_return.index, time_series_log_return.values, label='Brent Oil Log Returns', color='blue', alpha=0.7)

    # Plot the most probable change point
    plt.axvline(x=most_probable_tau_date_volatility, color='green', linestyle='-', linewidth=2, label='Most Probable Volatility Change Point (PyMC)')

    # Optionally, plot the posterior distribution of the change point
    # This shows the uncertainty around the change point location.
    # We map the indices back to dates for the histogram.
    plt.hist(time_series_log_return.index[tau_samples_volatility], bins=50, density=True, alpha=0.3, color='orange', label='Posterior Probability of Volatility Change Point')

    plt.title('Bayesian Change Point Detection (PyMC) on Brent Oil Log Returns (Volatility Shift)')
    plt.xlabel('Date')
    plt.ylabel('Log Return')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()
else:
    print("Skipping Bayesian Change Point Detection as log return data is not available or empty.")

In [ ]:
import matplotlib.pyplot as plt
import arviz as az
import pandas as pd
import numpy as np

# Assume these are already loaded or computed:
# time_series_price: pd.Series indexed by date with Brent oil prices
# detected_change_point_indices_ruptures: list of rupture change point indices (frequentist)
# trace_volatility: arviz.InferenceData from PyMC3 Bayesian model
# time_series_log_return: pd.Series indexed by date with log returns

# ------------------------
# 1. Plot Price with Rupture Change Points
plt.figure(figsize=(14,6))
plt.plot(time_series_price.index, time_series_price.values, label='Brent Oil Price')
for cp in detected_change_point_indices_ruptures:
    plt.axvline(time_series_price.index[cp], color='red', linestyle='--', alpha=0.7, label='Rupture Change Point' if cp == detected_change_point_indices_ruptures[0] else "")
plt.title('Brent Oil Price with Detected Change Points (Ruptures)')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# ------------------------
# 2. Plot Bayesian Posterior Distribution for tau (Change Point)
tau_samples = trace_volatility.posterior["tau"].values.flatten()
plt.figure(figsize=(12,5))
plt.hist(time_series_log_return.index[tau_samples.astype(int)], bins=50, density=True, color='orange', alpha=0.5)
plt.title('Posterior Distribution of Change Point (Bayesian Model)')
plt.xlabel('Date')
plt.ylabel('Probability Density')
plt.grid(True)
plt.tight_layout()
plt.show()

# ------------------------
# 3. Quantify Impact for Ruptures (Price Mean Shift)

if detected_change_point_indices_ruptures and len(detected_change_point_indices_ruptures) > 0:
    print("\n--- Ruptures (Price Mean Shift) Impact ---")
    segments = [0] + detected_change_point_indices_ruptures + [len(time_series_price)]
    for i in range(len(segments)-1):
        start_idx = segments[i]
        end_idx = segments[i+1]
        segment_data = time_series_price.iloc[start_idx:end_idx]
        if not segment_data.empty:
            end_date_idx = end_idx - 1 if end_idx > 0 else 0
            print(f"Segment {i+1} (Price): {time_series_price.index[start_idx].strftime('%Y-%m-%d')} to {time_series_price.index[end_date_idx].strftime('%Y-%m-%d')}")
            print(f"  Mean Price: {segment_data.mean():.2f} USD")
            print(f"  Std Dev Price: {segment_data.std():.2f} USD")
            if i > 0:
                prev_segment_data = time_series_price.iloc[segments[i-1]:segments[i]]
                if not prev_segment_data.empty:
                    prev_segment_mean = prev_segment_data.mean()
                    price_change = segment_data.mean() - prev_segment_mean
                    percent_change = (price_change / prev_segment_mean) * 100 if prev_segment_mean != 0 else float('inf')
                    print(f"  Change from previous segment: {price_change:.2f} USD ({percent_change:.2f}%)")
else:
    print("No rupture change points detected or data unavailable.")

# ------------------------
# 4. Quantify Impact for PyMC Bayesian Volatility Shift

print("\n--- Bayesian Volatility Shift Impact ---")
posterior_mu_log_return = trace_volatility.posterior["mu_log_return"].mean().item()
posterior_sigma_1 = trace_volatility.posterior["sigma_1"].mean().item()
posterior_sigma_2 = trace_volatility.posterior["sigma_2"].mean().item()

print(f"Estimated Mean Log Return (overall): {posterior_mu_log_return:.4f}")
print(f"Estimated Std Dev Before Change (sigma_1): {posterior_sigma_1:.4f}")
print(f"Estimated Std Dev After Change (sigma_2): {posterior_sigma_2:.4f}")

percent_change_volatility = ((posterior_sigma_2 - posterior_sigma_1) / posterior_sigma_1) * 100 if posterior_sigma_1 != 0 else float('inf')
print(f"Percentage change in volatility: {percent_change_volatility:.2f}%")

most_probable_tau_index = int(pd.Series(tau_samples).mode())
most_probable_tau_date = time_series_log_return.index[most_probable_tau_index]
print(f"Most Probable Change Point Index (Bayesian tau): {most_probable_tau_index}")
print(f"Most Probable Change Point Date (Bayesian tau): {most_probable_tau_date.strftime('%Y-%m-%d')}")

# ------------------------
# 5. Hypothesize Real-World Event Associations

print("\n--- Hypothesized Event Associations ---")
# Example: You should replace these dates and events with your Task 1 researched events
events_dict = {
    '1987-08-20': 'OPEC Production Cut Announcement',
    '1990-08-06': 'Gulf War Start',
    '2008-08-20': 'Global Financial Crisis Peaks',
    'Most probable Bayesian change point': most_probable_tau_date.strftime('%Y-%m-%d')
}

for date_str, event_desc in events_dict.items():
    print(f"Date: {date_str}, Event: {event_desc}")

# ------------------------
# 6. Convergence Diagnostics - Trace Plots and R-hat values

print("\n--- Convergence Diagnostics ---")
summary = az.summary(trace_volatility, var_names=["tau", "mu_log_return", "sigma_1", "sigma_2"])
print(summary[['mean','sd','r_hat','ess_bulk']])

az.plot_trace(trace_volatility, var_names=["tau", "mu_log_return", "sigma_1", "sigma_2"])
plt.tight_layout()
plt.show()


In [ ]:
import arviz as az
import matplotlib.pyplot as plt

# Plot trace for model diagnostics
az.plot_trace(trace_volatility, var_names=["tau", "mu_log_return", "sigma_1", "sigma_2"])
plt.tight_layout()
plt.show()

# Plot posterior distribution for change point
az.plot_posterior(trace_volatility, var_names=["tau"], hdi_prob=0.95)
plt.show()

# Optional: Plot mean and std dev segments side by side
segments = [0] + detected_change_point_indices_ruptures + [len(time_series_price)]
means = []
stds = []
for i in range(len(segments)-1):
    segment_data = time_series_price.iloc[segments[i]:segments[i+1]]
    means.append(segment_data.mean())
    stds.append(segment_data.std())

plt.figure(figsize=(12,5))
plt.errorbar(range(len(means)), means, yerr=stds, fmt='o', capsize=5)
plt.xlabel('Segment')
plt.ylabel('Mean Price with Std Dev')
plt.title('Segment-wise Mean Price and Volatility')
plt.show()


In [ ]:
import arviz as az

# Assuming your trace object is named 'trace_volatility'
summary_stats = az.summary(trace_volatility)
print(summary_stats[['r_hat']])


In [ ]:
pm.plot_trace(trace_volatility, var_names=["tau", "mu_log_return", "sigma_1", "sigma_2"])
plt.show()


In [ ]:
print("\n--- Conceptual Further Analysis ---")
print("For both frequentist and Bayesian approaches, the next steps would typically involve:")
print("a. Contextualizing: What real-world events or policy changes occurred around these dates?")
print("   - For example, if analyzing Brent oil prices, a change point might correspond to a major OPEC decision, a geopolitical conflict, or a global economic sanction.")
print("b. Quantifying Impact: Analyze the statistical properties (mean, variance, trend) of the segments")
print("   before and after each change point to understand the magnitude and nature of the shift.")

# Example of quantifying impact for ruptures (Price Mean Shift)
if 'detected_change_point_indices_ruptures' in locals() and len(detected_change_point_indices_ruptures) > 0 and 'time_series_price' in locals():
    print("\n--- Ruptures (Price Mean Shift) Impact ---")
    segments = [0] + detected_change_point_indices_ruptures + [len(time_series_price)]
    for i in range(len(segments) - 1):
        start_idx = segments[i]
        end_idx = segments[i+1]
        segment_data = time_series_price.iloc[start_idx:end_idx]
        if not segment_data.empty:
            # Ensure end_idx is within bounds when accessing index
            end_date_idx = end_idx - 1 if end_idx > 0 else 0
            print(f"Segment {i+1} (Price): {time_series_price.index[start_idx].strftime('%Y-%m-%d')} to {time_series_price.index[end_date_idx].strftime('%Y-%m-%d')}")
            print(f"  Mean Price: {segment_data.mean():.2f} USD")
            print(f"  Std Dev Price: {segment_data.std():.2f} USD")
            if i > 0:
                prev_segment_data = time_series_price.iloc[segments[i-1]:segments[i]]
                if not prev_segment_data.empty:
                    prev_segment_mean = prev_segment_data.mean()
                    if not pd.isna(prev_segment_mean):
                        price_change = segment_data.mean() - prev_segment_mean
                        percent_change = (price_change / prev_segment_mean) * 100 if prev_segment_mean != 0 else float('inf') # Handle division by zero
                        print(f"  Change from previous segment: {price_change:.2f} USD ({percent_change:.2f}%)")
else:
    print("\n--- Ruptures (Price Mean Shift) Impact ---")
    print("Skipping impact analysis for ruptures as change points were not detected or data is not available.")

# Example of quantifying impact for PyMC (Log Return Volatility Shift)
if 'trace_volatility' in locals():
    print("\n--- PyMC (Log Return Volatility Shift) Impact ---")
    posterior_mu_log_return = trace_volatility.posterior["mu_log_return"].mean().item()
    posterior_sigma_1 = trace_volatility.posterior["sigma_1"].mean().item()
    posterior_sigma_2 = trace_volatility.posterior["sigma_2"].mean().item()

    print(f"Estimated Mean Log Return (overall): {posterior_mu_log_return:.4f}")
    print(f"Estimated Std Dev of Log Returns before Change (sigma_1): {posterior_sigma_1:.4f}")
    print(f"Estimated Std Dev of Log Returns after Change (sigma_2): {posterior_sigma_2:.4f}")
    # Handle division by zero for percentage change
    percent_change_volatility = ((posterior_sigma_2 - posterior_sigma_1) / posterior_sigma_1) * 100 if posterior_sigma_1 != 0 else float('inf')
    print(f"Percentage change in volatility: {percent_change_volatility:.2f}%")
else:
    print("\n--- PyMC (Log Return Volatility Shift) Impact ---")
    print("Skipping impact analysis for PyMC as the Bayesian model was not run successfully.")


print("\nc. Modeling: Build separate models for each segment if the underlying process has truly changed.")
print("   This can lead to more accurate forecasts and better understanding of the system.")

In [ ]:
az.plot_posterior(trace_volatility, var_names=["tau"], hdi_prob=0.95)
plt.show()
